In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download amaldev007/driver-drowsiness-detection-system-dataset

Dataset URL: https://www.kaggle.com/datasets/amaldev007/driver-drowsiness-detection-system-dataset
License(s): unknown
 96% 2.08G/2.17G [00:27<00:03, 24.8MB/s]
100% 2.17G/2.17G [00:27<00:00, 84.1MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/driver-drowsiness-detection-system-dataset.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling openc

In [ ]:
import os
import cv2
import mediapipe as mp
from tqdm import tqdm
from multiprocessing import Pool, Manager, cpu_count
import time

mp_face_detection = mp.solutions.face_detection

def process_image(args):
    img_path, class_name, img_name, output_dir = args
    try:
        image = cv2.imread(img_path)
        if image is None:
            return f"Skipping unreadable image: {img_path}"

        height, width, _ = image.shape

        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
            results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.detections:
            detection = results.detections[0]
            bboxC = detection.location_data.relative_bounding_box
            x_min = int(bboxC.xmin * width)
            y_min = int(bboxC.ymin * height)
            box_width = int(bboxC.width * width)
            box_height = int(bboxC.height * height)

            x_min = max(0, x_min)
            y_min = max(0, y_min)

            cropped_face = image[y_min:y_min+box_height, x_min:x_min+box_width]

            if cropped_face.size == 0:
                return f"Empty crop for {img_path}, skipping."

            cropped_face = cv2.resize(cropped_face, (256, 256))

            class_output_path = os.path.join(output_dir, class_name)
            if not os.path.exists(class_output_path):
                os.makedirs(class_output_path)
            output_path = os.path.join(class_output_path, img_name)
            cv2.imwrite(output_path, cropped_face)
            return f"Cropped and saved: {output_path}"
        else:
            return f"No face detected in: {img_path}"

    except Exception as e:
        return f"Error processing {img_path}: {e}"


def crop_faces_multiprocessing(input_dir, output_dir, num_workers=None):
    if num_workers is None:
        num_workers = max(1, cpu_count() - 1)

    all_images = []
    for class_name in os.listdir(input_dir):
        class_input_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_input_path):
            continue
        for img_name in os.listdir(class_input_path):
            img_path = os.path.join(class_input_path, img_name)
            all_images.append((img_path, class_name, img_name, output_dir))

    total_images = len(all_images)
    print(f"Total images to process: {total_images}")
    start_time = time.time()

    with Pool(num_workers) as pool:

        results = list(tqdm(pool.imap(process_image, all_images), total=total_images, desc="Cropping faces"))

    end_time = time.time()
    elapsed = end_time - start_time
    print(f"\nProcessing completed in {elapsed:.2f} seconds ({elapsed/60:.2f} minutes).")


input_dataset_path = '/content/0 FaceImages'
output_dataset_path = '/content/cleaned_dataset'

crop_faces_multiprocessing(input_dataset_path, output_dataset_path)


Total images to process: 9120


Cropping faces: 100%|██████████| 9120/9120 [02:59<00:00, 50.89it/s]


Processing completed in 179.26 seconds (2.99 minutes).


In [ ]:
!pip install split-folders

In [ ]:
import os
import splitfolders

input_dir = "/content/cleaned_dataset"
output_dir = '/content/split_data'


os.makedirs(output_dir, exist_ok=True)


input_dir_extracted = "/content/cleaned_dataset"

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, seed=1337, ratio=(.8, .2), move=False)

Copying files: 9119 files [00:01, 6320.30 files/s]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping=EarlyStopping(monitor='val_accuracy',patience=4,restore_best_weights=True)

In [ ]:
train_ds=keras.utils.image_dataset_from_directory(
    directory="/content/split_data/train",
    labels="inferred",
    label_mode="int",
    batch_size=16,
    image_size=(224,224)
)

test_ds=keras.utils.image_dataset_from_directory(
    directory="/content/split_data/val",
    labels="inferred",
    label_mode="int",
    batch_size=16,
    image_size=(224,224)
)

Found 7295 files belonging to 2 classes.
Found 1824 files belonging to 2 classes.


In [ ]:
def process(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
    horizontal_flip=True,
    rescale=1./255)



train_generator = datagen.flow_from_directory(
    '/content/split_data/train',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=16,
    class_mode='binary',
    shuffle=True
)

test_generator=datagen.flow_from_directory(
    '/content/split_data/val',
    target_size=(224, 224),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=16)

Found 7295 images belonging to 2 classes.
Found 1824 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu',kernel_regularizer=l2(0.001), input_shape=(224, 224, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu',kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu',kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(train_ds,epochs=10,validation_data=test_ds,callbacks=early_stopping)

Epoch 1/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.6502 - loss: 0.7629 - val_accuracy: 0.8333 - val_loss: 0.4709
Epoch 2/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.8304 - loss: 0.4675 - val_accuracy: 0.8909 - val_loss: 0.3737
Epoch 3/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.8725 - loss: 0.3976 - val_accuracy: 0.9156 - val_loss: 0.3221
Epoch 4/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.8997 - loss: 0.3503 - val_accuracy: 0.9265 - val_loss: 0.2990
Epoch 5/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.9102 - loss: 0.3292 - val_accuracy: 0.9293 - val_loss: 0.2910
Epoch 6/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.9156 - loss: 0.3065 - val_accuracy: 0.9402 - val_loss: 0.2587
Epoch 7/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9264 - loss: 0.2878 - val_accuracy: 0.9457 - val_loss: 0.2555
Epoch 8/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9317 - loss: 0.2780 - 

In [ ]:
model.save("new_driver_drowsiness7.h5")
from google.colab import files
files.download("new_driver_drowsiness7.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>